In [ ]:
!pip install roboflow
!pip install ultralytics

In [ ]:
import os
import cv2
from ultralytics import YOLO
from roboflow import Roboflow
import matplotlib.pyplot as plt

In [ ]:
rf = Roboflow(api_key="oYlcM7miU5zQhoyMjcAo")
project = rf.workspace("youssef-elsheikh-pqoup").project("facial-emotion-recognition-hnujs")
version = project.version(1)
dataset = version.download("yolov11")

In [ ]:
print(dataset.location)

In [ ]:
yaml_content = """
test: /kaggle/working/Facial-emotion-recognition-1/test
train: /kaggle/working/Facial-emotion-recognition-1/train
val: /kaggle/working/Facial-emotion-recognition-1/valid

nc: 4
names:
  - angry
  - happy
  - sad
  - surprised
"""

with open('/kaggle/working/Facial-emotion-recognition-1/data1.yaml', 'w') as f:
    f.write(yaml_content)

In [ ]:
fine_model = YOLO('yolo11n.pt')
normal_model = YOLO('yolo11n.pt')
transfer_model = YOLO('yolo11n.pt')

In [ ]:
fine_model = YOLO('fine_tuned_yolo.pt')
normal_model = YOLO('yolo11n.pt')
transfer_model = YOLO('transfer_yolo.pt')

In [ ]:
fine_results = fine_model.train(data='/kaggle/working/Facial-emotion-recognition-1/data1.yaml', epochs=25, patience=5 , project='Facial Emotion Fine-tune', name='logs', device=[0])

In [ ]:
fine_model.save("fine_tuned_yolo.pt")

In [ ]:
transfer_results = transfer_model.train(data='/kaggle/working/Facial-emotion-recognition-1/data1.yaml', epochs=25 ,patience=5, project='Facial Emotion transfer', name='logs', device=[0] , freeze="backbone")

In [ ]:
transfer_model.save("transfer_yolo.pt")

In [ ]:
import cv2
import matplotlib.pyplot as plt

# Load images
img_fine = cv2.imread("/kaggle/working/Facial Emotion Fine-tune/logs/results.png")
img_transfer = cv2.imread("/kaggle/working/Facial Emotion transfer/logs2/results.png")

# Convert BGR to RGB
img_fine = cv2.cvtColor(img_fine, cv2.COLOR_BGR2RGB)
img_transfer = cv2.cvtColor(img_transfer, cv2.COLOR_BGR2RGB)

# Create side-by-side plot
plt.figure(figsize=(14,6))  # width x height

plt.subplot(1, 2, 1)  # 1 row, 2 columns, first plot
plt.imshow(img_fine)
plt.axis("off")
plt.title("Fine-tuned YOLO")

plt.subplot(1, 2, 2)  # 1 row, 2 columns, second plot
plt.imshow(img_transfer)
plt.axis("off")
plt.title("Transfer-learned YOLO")

plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load CSVs and titles
dfs = [
    ("/kaggle/working/Facial Emotion Fine-tune/logs/results.csv", "Fine-tuned YOLO"),
    ("/kaggle/working/Facial Emotion transfer/logs2/results.csv", "Transfer-learned YOLO")
]

# Loss types
losses = ["box", "cls", "dfl"]

plt.figure(figsize=(18, 5))

for idx, loss in enumerate(losses, 1):
    plt.subplot(1, 3, idx)

    for path, title in dfs:
        df = pd.read_csv(path)

        # Training loss
        col_train = f"train/{loss}_loss"
        plt.plot(df["epoch"], df[col_train], label=f"{title} Train")

        # Validation loss
        col_val = f"val/{loss}_loss"
        if col_val in df.columns:
            plt.plot(df["epoch"], df[col_val], linestyle="--", label=f"{title} Val")

    plt.xlabel("Epoch")
    plt.ylabel(f"{loss.upper()} Loss")
    plt.title(f"{loss.upper()} Loss Comparison")
    plt.legend()
    plt.grid(True)

plt.tight_layout()
plt.show()


In [ ]:
# Evaluate Fine-Tune model
fine_test = fine_model.val(data='/kaggle/working/Facial-emotion-recognition-1/data1.yaml', split="test")

# Evaluate original model
normal_test = normal_model.val(data='/kaggle/working/Facial-emotion-recognition-1/data1.yaml', split="test")

# Evaluate Transfer model
transfer_test = transfer_model.val(data='/kaggle/working/Facial-emotion-recognition-1/data1.yaml', split="test")

In [ ]:
# Extract main metrics from a YOLOv11 DetMetrics object
def extract_metrics(result):
    # mean_results() returns precision, recall, mAP50, mAP50-95
    precision, recall, mAP50, mAP5095 = result.mean_results()
    return precision, recall, mAP50, mAP5095

# Fine-tuned model
fine_prec, fine_rec, fine_mAP50, fine_mAP5095 = extract_metrics(fine_test)

# Original model
norm_prec, norm_rec, norm_mAP50, norm_mAP5095 = extract_metrics(normal_test)

# Transfer-learned model
trans_prec, trans_rec, trans_mAP50, trans_mAP5095 = extract_metrics(transfer_test)

In [ ]:
import matplotlib.pyplot as plt

models = ["Fine-tuned YOLO", "Original YOLO", "Transfer-learned YOLO"]
mAP50 = [fine_mAP50, norm_mAP50, trans_mAP50]
mAP5095 = [fine_mAP5095, norm_mAP5095, trans_mAP5095]

x = range(len(models))
width = 0.20

plt.figure(figsize=(8,5))
plt.bar(x, mAP50, width=width, label="mAP@0.5")
plt.bar([i + width for i in x], mAP5095, width=width, label="mAP@0.5:0.95")
plt.xticks([i + width/2 for i in x], models, rotation=15)
plt.ylabel("mAP")
plt.title("Model Comparison on Test Set")
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(axis="y")
plt.show()


In [ ]:
import cv2
import matplotlib.pyplot as plt
import os
import random

# Folder containing test images
test_folder = "/kaggle/working/Facial-emotion-recognition-1/test/images"
all_images = [os.path.join(test_folder, f) for f in os.listdir(test_folder) if f.endswith((".jpg", ".png"))]

# Pick 1 random image
img_path = random.choice(all_images)

# Models
models = [
    (fine_model, "Fine-tuned YOLO"),
    (normal_model, "Original YOLO"),
    (transfer_model, "Transfer-learned YOLO")
]

# Plot
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
fig.suptitle("Predictions of 3 Models on One Random Test Image", fontsize=16)

for i, (model, model_name) in enumerate(models):
    # Run prediction
    results = model.predict(img_path)
    pred_img = results[0].plot()  # prediction with boxes

    # Plot
    axes[i].imshow(pred_img)
    axes[i].axis("off")
    axes[i].set_title(model_name, fontsize=12)

plt.tight_layout()
plt.subplots_adjust(top=0.85)
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load CSVs and titles
dfs = [
    ("/kaggle/input/10-epochs/fine_results.csv", "Fine-tuned YOLO"),
    ("/kaggle/input/10-epochs/transfer_results.csv", "Transfer-learned YOLO")
]

# Loss types
losses = ["box", "cls", "dfl"]

plt.figure(figsize=(18, 5))

for idx, loss in enumerate(losses, 1):
    plt.subplot(1, 3, idx)

    for path, title in dfs:
        df = pd.read_csv(path)

        # Training loss
        col_train = f"train/{loss}_loss"
        plt.plot(df["epoch"], df[col_train], label=f"{title} Train")

        # Validation loss
        col_val = f"val/{loss}_loss"
        if col_val in df.columns:
            plt.plot(df["epoch"], df[col_val], linestyle="--", label=f"{title} Val")

    plt.xlabel("Epoch")
    plt.ylabel(f"{loss.upper()} Loss")
    plt.title(f"{loss.upper()} Loss Comparison")
    plt.legend()
    plt.grid(True)

plt.tight_layout()
plt.show()
